# 🏦 Investment Portfolio Analyzer
# Complete Walkthrough: Phase 1 & Phase 2

---

## Welcome!

This notebook is a **comprehensive, beginner-friendly guide** to everything we've built. We'll cover:

### 📦 Phase 1: Foundation
- Docker & containerization
- Project structure
- Configuration management
- Database setup (PostgreSQL + SQLAlchemy)
- Data models (tables)
- Pydantic schemas (validation)
- FastAPI routers (endpoints)

### 📊 Phase 2: Data Ingestion
- Market data integration (Yahoo Finance)
- Asset resolution service
- Transaction management
- Batch operations
- CSV file upload

---

## 🖥️ CRITICAL: View Logs Side-by-Side!

**Open a separate terminal** and run:

```bash
docker compose logs -f backend
```

Keep this visible as you work through the notebook:

```
┌─────────────────────────────────┬─────────────────────────────────┐
│         JUPYTER NOTEBOOK        │       TERMINAL (Logs)           │
│                                 │                                 │
│   [Run cells here]              │   docker compose logs -f backend│
│                                 │                                 │
│   Watch the output...           │   See what the server does...   │
└─────────────────────────────────┴─────────────────────────────────┘
```

---

## 📋 Table of Contents

**PHASE 1: FOUNDATION**
1. [Prerequisites & Setup](#1-prerequisites)
2. [Understanding Docker](#2-docker)
3. [Project Structure](#3-project-structure)
4. [Configuration Management](#4-configuration)
5. [Database Connection](#5-database-connection)
6. [SQLAlchemy Models](#6-models)
7. [Pydantic Schemas](#7-schemas)
8. [FastAPI Routers](#8-routers)
9. [Starting the Server](#9-starting-server)

**PHASE 2: DATA INGESTION**
10. [Yahoo Finance Integration](#10-yahoo)
11. [Asset Resolution Service](#11-asset-resolution)
12. [Creating Transactions](#12-transactions)
13. [Batch Operations](#13-batch)
14. [CSV File Upload](#14-csv-upload)
15. [Error Handling](#15-errors)

**EXPLORATION**
16. [Data Exploration](#16-exploration)
17. [Debugging & Logs](#17-debugging)
18. [Your Experiments](#18-experiments)

---

---

# 🔷 PHASE 1: FOUNDATION

---

<a id='1-prerequisites'></a>
# 1️⃣ Prerequisites & Setup

Before we begin, let's make sure you have everything installed.

## What You Need

| Software | Purpose | Check Command |
|----------|---------|---------------|
| **Docker Desktop** | Runs containers | `docker --version` |
| **Docker Compose** | Orchestrates multiple containers | `docker compose version` |
| **Python 3.10+** | Runs this notebook | `python --version` |
| **Git** | Version control | `git --version` |

### Install Links (if needed)
- Docker Desktop: https://www.docker.com/products/docker-desktop/
- Python: https://www.python.org/downloads/

In [ ]:
# Let's verify your setup

import subprocess
import sys
import shutil

def check_command(name, cmd):
    """Check if a command exists and get its version."""
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            version = result.stdout.strip().split('\n')[0]
            print(f"✅ {name}: {version}")
            return True
        else:
            print(f"❌ {name}: Not found or error")
            return False
    except Exception as e:
        print(f"❌ {name}: Error - {e}")
        return False

print("Checking prerequisites...\n")
print("=" * 50)

checks = [
    ("Docker", "docker --version"),
    ("Docker Compose", "docker compose version"),
    ("Python", f"{sys.executable} --version"),
    ("Git", "git --version"),
]

all_good = all(check_command(name, cmd) for name, cmd in checks)

print("=" * 50)
if all_good:
    print("\n🎉 All prerequisites are installed!")
else:
    print("\n⚠️ Some prerequisites are missing. Please install them before continuing.")

In [ ]:
# Install Python packages needed for this notebook

def install_package(package, import_name=None):
    """Install a package if not already installed."""
    import_name = import_name or package
    try:
        __import__(import_name)
        print(f"✅ {package}")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "-q"])
        print(f"✅ {package} installed")

print("Checking Python packages...\n")

packages = [
    ("requests", None),
    ("pandas", None),
    ("ipywidgets", None),
    ("tabulate", None),
]

for pkg, imp in packages:
    install_package(pkg, imp)

print("\n🎉 All Python packages ready!")

In [ ]:
# Import all libraries we'll use throughout this notebook

import requests
import pandas as pd
import json
import time
import os
import tempfile
from datetime import datetime, timedelta
from pathlib import Path
from IPython.display import display, HTML, Markdown, Code, clear_output
import ipywidgets as widgets

# Configure pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# API URL (FastAPI server in Docker)
API_URL = "http://localhost:8000"

print("✅ All libraries imported!")
print(f"🌐 API URL: {API_URL}")

In [ ]:
# Helper functions for nice output (used throughout the notebook)

def show_success(text):
    display(HTML(f'<div style="background:#e8f5e9;border-left:4px solid #4CAF50;padding:10px;margin:10px 0;">✅ {text}</div>'))

def show_error(text):
    display(HTML(f'<div style="background:#ffebee;border-left:4px solid #f44336;padding:10px;margin:10px 0;">❌ {text}</div>'))

def show_info(text):
    display(HTML(f'<div style="background:#e3f2fd;border-left:4px solid #2196F3;padding:10px;margin:10px 0;">ℹ️ {text}</div>'))

def show_warning(text):
    display(HTML(f'<div style="background:#fff3e0;border-left:4px solid #FF9800;padding:10px;margin:10px 0;">⚠️ {text}</div>'))

def show_code(code, language="python"):
    display(HTML(f'<pre style="background:#2d2d2d;color:#f8f8f2;padding:15px;border-radius:8px;overflow-x:auto;">{code}</pre>'))

def show_response(response, title="Response"):
    status_emoji = "✅" if response.status_code < 400 else "❌"
    try:
        body = json.dumps(response.json(), indent=2, default=str)
    except:
        body = response.text
    display(HTML(f'''
    <div style="background:#f8f9fa;padding:15px;border-radius:8px;margin:10px 0;">
        <h4 style="margin-top:0;">{title}</h4>
        <p><strong>Status:</strong> {status_emoji} {response.status_code}</p>
        <pre style="background:#2d2d2d;color:#f8f8f2;padding:10px;border-radius:4px;overflow-x:auto;max-height:400px;">{body}</pre>
    </div>
    '''))

def run_command(cmd, show_output=True):
    """Run a shell command."""
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if show_output and result.stdout:
        print(result.stdout)
    if result.stderr and result.returncode != 0:
        print(f"Error: {result.stderr}")
    return result

def timed_request(method, url, **kwargs):
    """Make a timed HTTP request."""
    start = time.time()
    response = requests.request(method, url, **kwargs)
    elapsed = time.time() - start
    return response, elapsed

print("✅ Helper functions ready!")

<a id='2-docker'></a>
# 2️⃣ Understanding Docker

Docker is the foundation of our development environment. Let's understand it thoroughly.

## What is Docker?

Docker creates **isolated containers** that package your application with all its dependencies.

```
┌─────────────────────────────────────────────────────────────────┐
│                      YOUR COMPUTER                              │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐ │
│  │   Container 1   │  │   Container 2   │  │   Container 3   │ │
│  │   (Backend)     │  │   (Database)    │  │   (Frontend)    │ │
│  │                 │  │                 │  │                 │ │
│  │  Python 3.11    │  │  PostgreSQL 16  │  │  Node.js 20     │ │
│  │  FastAPI        │  │  Your Data      │  │  React          │ │
│  │  Dependencies   │  │                 │  │  Dependencies   │ │
│  └─────────────────┘  └─────────────────┘  └─────────────────┘ │
│           │                    │                    │          │
│           └────────────────────┴────────────────────┘          │
│                              │                                  │
│                    Docker Network                               │
│                 (containers can talk)                           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Why Docker?

| Benefit | Explanation |
|---------|-------------|
| **Consistency** | Same environment on every machine |
| **Isolation** | Dependencies don't conflict |
| **Reproducibility** | "Works on my machine" → "Works everywhere" |
| **Easy cleanup** | Delete container = clean slate |

## Key Docker Files

### 1. `Dockerfile` - How to Build an Image

```dockerfile
# Start from Python 3.11 base image
FROM python:3.11-slim

# Set working directory inside container
WORKDIR /app

# Copy requirements first (for caching)
COPY requirements.txt .

# Install Python dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY . .

# Command to run when container starts
CMD ["uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8000", "--reload"]
```

**What each line does:**
- `FROM`: Base image (pre-built Python environment)
- `WORKDIR`: Create and switch to /app directory
- `COPY`: Copy files from your computer into container
- `RUN`: Execute commands during build
- `CMD`: Default command when container starts

### 2. `docker-compose.yml` - Orchestrate Multiple Containers

```yaml
version: '3.8'

services:
  # PostgreSQL Database
  db:
    image: postgres:16
    environment:
      POSTGRES_USER: portfolio_user
      POSTGRES_PASSWORD: portfolio_pass
      POSTGRES_DB: portfolio_db
    volumes:
      - postgres_data:/var/lib/postgresql/data
    ports:
      - "5432:5432"

  # FastAPI Backend
  backend:
    build: ./backend
    ports:
      - "8000:8000"
    volumes:
      - ./backend:/app
    environment:
      DATABASE_URL: postgresql://portfolio_user:portfolio_pass@db:5432/portfolio_db
    depends_on:
      - db

volumes:
  postgres_data:
```

**Key concepts:**
- `services`: Each container definition
- `image`: Use pre-built image (postgres)
- `build`: Build from Dockerfile
- `ports`: Map container port to host port
- `volumes`: Persist data / sync code
- `depends_on`: Start order
- `environment`: Pass variables to container

## Essential Docker Commands

| Command | What it does |
|---------|-------------|
| `docker compose up -d` | Start containers in background |
| `docker compose down` | Stop and remove containers |
| `docker compose logs -f backend` | Follow backend logs |
| `docker compose ps` | List running containers |
| `docker compose restart backend` | Restart a service |
| `docker compose exec backend bash` | Open shell in container |
| `docker compose build` | Rebuild images |

In [ ]:
# Let's check Docker status

display(Markdown("### 🐳 Docker Status"))

# Check if Docker daemon is running
result = run_command("docker info", show_output=False)
if result.returncode == 0:
    show_success("Docker daemon is running")
else:
    show_error("Docker daemon is NOT running. Please start Docker Desktop!")

In [ ]:
# Check current container status

display(Markdown("### 📦 Container Status"))

result = run_command("docker compose ps", show_output=True)

if "backend" in result.stdout and "running" in result.stdout.lower():
    show_success("Containers are running!")
else:
    show_warning("Containers may not be running. We'll start them in Section 9.")

<a id='3-project-structure'></a>
# 3️⃣ Project Structure

Let's understand how our codebase is organized.

## Directory Layout

```
investment-portfolio-analyzer/
│
├── 📄 docker-compose.yml      # Container orchestration
├── 📄 README.md               # Project documentation
│
└── 📁 backend/                # Python backend application
    │
    ├── 📄 Dockerfile          # How to build backend image
    ├── 📄 requirements.txt    # Python dependencies
    ├── 📄 .env                # Environment variables (secrets!)
    ├── 📄 init_db.py          # Database initialization script
    │
    └── 📁 app/                # Application code
        │
        ├── 📄 __init__.py     # Package marker
        ├── 📄 main.py         # FastAPI app entry point
        ├── 📄 config.py       # Configuration management
        ├── 📄 database.py     # Database connection
        ├── 📄 models.py       # SQLAlchemy models (tables)
        ├── 📄 utils.py        # Utility functions (logging)
        │
        ├── 📁 schemas/        # Pydantic validation schemas
        │   ├── __init__.py
        │   ├── assets.py
        │   ├── portfolios.py
        │   ├── transactions.py
        │   └── upload.py
        │
        ├── 📁 routers/        # API endpoint handlers
        │   ├── __init__.py
        │   ├── assets.py
        │   ├── portfolios.py
        │   ├── transactions.py
        │   └── upload.py
        │
        └── 📁 services/       # Business logic
            ├── __init__.py
            ├── asset_resolution.py
            │
            ├── 📁 market_data/
            │   ├── base.py    # Abstract provider
            │   └── yahoo.py   # Yahoo Finance
            │
            └── 📁 upload/
                ├── service.py
                └── 📁 parsers/
                    ├── base.py
                    └── csv_parser.py
```

## Layer Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                         CLIENT                                  │
│              (This notebook, Browser, Mobile App)               │
└───────────────────────────┬─────────────────────────────────────┘
                            │ HTTP (JSON)
                            ▼
┌─────────────────────────────────────────────────────────────────┐
│                      ROUTERS (routers/)                         │
│   Handle HTTP requests, validate input, return responses        │
│   • assets.py, portfolios.py, transactions.py, upload.py       │
└───────────────────────────┬─────────────────────────────────────┘
                            │ Function calls
                            ▼
┌─────────────────────────────────────────────────────────────────┐
│                     SERVICES (services/)                        │
│   Business logic, external API calls, complex operations        │
│   • asset_resolution.py, market_data/, upload/                 │
└───────────────────────────┬─────────────────────────────────────┘
                            │ ORM queries
                            ▼
┌─────────────────────────────────────────────────────────────────┐
│                      MODELS (models.py)                         │
│   SQLAlchemy ORM - Python classes that map to database tables   │
│   • User, Portfolio, Asset, Transaction, MarketData            │
└───────────────────────────┬─────────────────────────────────────┘
                            │ SQL
                            ▼
┌─────────────────────────────────────────────────────────────────┐
│                      DATABASE (PostgreSQL)                      │
│   Persistent storage - tables, indexes, constraints             │
└─────────────────────────────────────────────────────────────────┘
```

<a id='4-configuration'></a>
# 4️⃣ Configuration Management

How we manage settings and secrets.

## The `.env` File

**Location:** `backend/.env`

```bash
# Database Connection
DATABASE_URL=postgresql://portfolio_user:portfolio_pass@db:5432/portfolio_db

# Application Settings
APP_NAME="Investment Portfolio Analyzer"
DEBUG=true
LOG_LEVEL=INFO

# Security (for future auth)
SECRET_KEY=your-super-secret-key-change-in-production
```

**⚠️ IMPORTANT:** Never commit `.env` to Git! It contains secrets.

## The `config.py` File

This file reads the `.env` and provides type-safe settings:

```python
# backend/app/config.py

from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    """Application settings loaded from environment."""
    
    # Database
    database_url: str
    
    # Application
    app_name: str = "Investment Portfolio Analyzer"
    debug: bool = False
    log_level: str = "INFO"
    
    # Security
    secret_key: str = "change-me"
    
    class Config:
        env_file = ".env"  # Load from .env file
        case_sensitive = False  # DATABASE_URL = database_url

# Global settings instance
settings = Settings()
```

**Usage anywhere in the app:**
```python
from app.config import settings

print(settings.app_name)  # "Investment Portfolio Analyzer"
print(settings.log_level)  # "INFO"
```

<a id='5-database-connection'></a>
# 5️⃣ Database Connection

How Python connects to PostgreSQL.

## The `database.py` File

```python
# backend/app/database.py

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base

from app.config import settings

# Create database engine (connection pool)
engine = create_engine(
    settings.database_url,
    pool_pre_ping=True,  # Check connection health
)

# Session factory (creates database sessions)
SessionLocal = sessionmaker(
    autocommit=False,
    autoflush=False,
    bind=engine
)

# Base class for all models
Base = declarative_base()

# Dependency for FastAPI (provides session per request)
def get_db():
    """Yield a database session, then close it."""
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()
```

### How It Works

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   Request   │────▶│  get_db()   │────▶│   Session   │
│   comes in  │     │  creates    │     │   object    │
└─────────────┘     └─────────────┘     └──────┬──────┘
                                               │
                                               ▼
                                        ┌─────────────┐
                                        │   Router    │
                                        │   uses it   │
                                        └──────┬──────┘
                                               │
                                               ▼
┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│  Response   │◀────│  get_db()   │◀────│   Session   │
│   sent      │     │  closes     │     │   closed    │
└─────────────┘     └─────────────┘     └─────────────┘
```

<a id='6-models'></a>
# 6️⃣ SQLAlchemy Models

Models define the structure of our database tables.

## Entity Relationship Diagram

```
┌─────────────────┐
│      USER       │
├─────────────────┤         ┌─────────────────┐
│ id (PK)         │────┐    │    PORTFOLIO    │
│ email           │    │    ├─────────────────┤
│ hashed_password │    └───▶│ id (PK)         │
│ is_active       │         │ user_id (FK)    │────┐
│ created_at      │         │ name            │    │
└─────────────────┘         │ currency        │    │
                            │ created_at      │    │
                            └─────────────────┘    │
                                                   │
┌─────────────────┐         ┌─────────────────┐    │
│      ASSET      │         │   TRANSACTION   │    │
├─────────────────┤         ├─────────────────┤    │
│ id (PK)         │◀────────│ id (PK)         │    │
│ ticker          │         │ portfolio_id(FK)│◀───┘
│ exchange        │         │ asset_id (FK)   │
│ name            │         │ transaction_type│
│ asset_class     │         │ date            │
│ sector          │         │ quantity        │
│ currency        │         │ price_per_share │
│ data_source     │         │ currency        │
│ proxy_asset_id  │         │ fee             │
│ is_active       │         │ created_at      │
└─────────────────┘         └─────────────────┘
        │
        │
        ▼
┌─────────────────┐
│   MARKET_DATA   │
├─────────────────┤
│ id (PK)         │
│ asset_id (FK)   │
│ date            │
│ open, high, low │
│ close, volume   │
│ is_synthetic    │
│ provider        │
└─────────────────┘
```

## Model Code Examples

### User Model

```python
class User(Base):
    """User account for authentication."""
    
    __tablename__ = "users"
    
    # Columns
    id: Mapped[int] = mapped_column(primary_key=True)
    email: Mapped[str] = mapped_column(String(255), unique=True, index=True)
    hashed_password: Mapped[str] = mapped_column(String(255))
    is_active: Mapped[bool] = mapped_column(Boolean, default=True)
    created_at: Mapped[datetime] = mapped_column(default=func.now())
    
    # Relationships
    portfolios: Mapped[list["Portfolio"]] = relationship(back_populates="user")
```

### Asset Model

```python
class Asset(Base):
    """A tradeable asset (stock, ETF, etc.)."""
    
    __tablename__ = "assets"
    
    id: Mapped[int] = mapped_column(primary_key=True)
    ticker: Mapped[str] = mapped_column(String(10), index=True)
    exchange: Mapped[str] = mapped_column(String(10), index=True)
    name: Mapped[str | None] = mapped_column(String(255))
    asset_class: Mapped[AssetClass] = mapped_column(SQLEnum(AssetClass))
    sector: Mapped[str | None] = mapped_column(String(100))
    currency: Mapped[str] = mapped_column(String(3))
    data_source: Mapped[str | None] = mapped_column(String(50))
    
    # Proxy backcasting (Phase 3)
    proxy_asset_id: Mapped[int | None] = mapped_column(ForeignKey("assets.id"))
    proxy_notes: Mapped[str | None] = mapped_column(String)
    
    # Unique constraint: one asset per ticker+exchange combination
    __table_args__ = (
        UniqueConstraint('ticker', 'exchange', name='uq_asset_ticker_exchange'),
    )
```

### Transaction Model

```python
class Transaction(Base):
    """A buy or sell transaction."""
    
    __tablename__ = "transactions"
    
    id: Mapped[int] = mapped_column(primary_key=True)
    portfolio_id: Mapped[int] = mapped_column(ForeignKey("portfolios.id"))
    asset_id: Mapped[int] = mapped_column(ForeignKey("assets.id"))
    
    transaction_type: Mapped[TransactionType] = mapped_column(SQLEnum(TransactionType))
    date: Mapped[datetime] = mapped_column(index=True)
    
    # Financial fields use Decimal for precision!
    quantity: Mapped[Decimal] = mapped_column(Numeric(18, 8))
    price_per_share: Mapped[Decimal] = mapped_column(Numeric(18, 8))
    currency: Mapped[str] = mapped_column(String(3))
    fee: Mapped[Decimal] = mapped_column(Numeric(18, 8), default=0)
```

<a id='7-schemas'></a>
# 7️⃣ Pydantic Schemas

Schemas validate API input/output.

## What Are Schemas?

```
┌─────────────────────────────────────────────────────────────────┐
│                    SCHEMA PURPOSES                              │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  📥 INPUT VALIDATION (Request)                                  │
│  ┌─────────────────┐    ┌─────────────────┐                    │
│  │ JSON from user  │───▶│ Pydantic Schema │───▶ Valid or Error │
│  │ {               │    │ TransactionCreate│                    │
│  │   "ticker": "X",│    │ - validates types│                    │
│  │   "qty": "10"   │    │ - converts values│                    │
│  │ }               │    │ - applies rules  │                    │
│  └─────────────────┘    └─────────────────┘                    │
│                                                                 │
│  📤 OUTPUT FORMATTING (Response)                                │
│  ┌─────────────────┐    ┌─────────────────┐                    │
│  │ SQLAlchemy Model│───▶│ Pydantic Schema │───▶ Clean JSON     │
│  │ Transaction obj │    │ TransactionResp │                    │
│  │ (from database) │    │ - selects fields│                    │
│  │                 │    │ - formats dates │                    │
│  └─────────────────┘    └─────────────────┘                    │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

## Schema Examples

### TransactionCreate (Input)

```python
class TransactionCreate(BaseModel):
    """Schema for creating a new transaction."""
    
    portfolio_id: int = Field(..., gt=0)  # Required, must be > 0
    ticker: str = Field(..., min_length=1, max_length=10)
    exchange: str = Field(..., min_length=0, max_length=10)
    transaction_type: TransactionType  # Enum: BUY or SELL
    
    date: datetime
    quantity: Decimal = Field(..., gt=0)  # Must be positive
    price_per_share: Decimal = Field(..., gt=0)
    currency: str = Field(default="EUR", pattern=r"^[A-Z]{3}$")
    fee: Decimal = Field(default=Decimal("0"), ge=0)
    
    @field_validator('ticker')
    @classmethod
    def normalize_ticker(cls, v):
        return v.strip().upper()  # "nvda" → "NVDA"
```

### TransactionResponse (Output)

```python
class TransactionResponse(BaseModel):
    """Schema for API responses."""
    
    id: int
    portfolio_id: int
    asset_id: int
    transaction_type: TransactionType
    date: datetime
    quantity: Decimal
    price_per_share: Decimal
    currency: str
    fee: Decimal
    created_at: datetime
    
    asset: AssetResponse  # Nested asset details
    
    model_config = ConfigDict(from_attributes=True)  # Allow ORM objects
```

<a id='8-routers'></a>
# 8️⃣ FastAPI Routers

Routers define API endpoints.

## Router Structure

```python
# backend/app/routers/transactions.py

from fastapi import APIRouter, Depends, HTTPException, status
from sqlalchemy.orm import Session

from app.database import get_db
from app.schemas.transactions import TransactionCreate, TransactionResponse
from app.services.asset_resolution import AssetResolutionService

# Create router with prefix and tags
router = APIRouter(
    prefix="/transactions",
    tags=["Transactions"]
)

@router.post("/", response_model=TransactionResponse, status_code=201)
def create_transaction(
    transaction: TransactionCreate,  # Pydantic validates input
    db: Session = Depends(get_db),   # Inject database session
):
    """
    Create a new transaction.
    
    - Resolves asset (creates if needed)
    - Validates portfolio exists
    - Creates transaction record
    """
    # Business logic here...
    return db_transaction


@router.get("/", response_model=TransactionListResponse)
def list_transactions(
    portfolio_id: int | None = None,  # Optional filter
    skip: int = 0,
    limit: int = 50,
    db: Session = Depends(get_db),
):
    """List transactions with optional filtering."""
    # Query logic here...
    return {"items": transactions, "total": count}
```

## Router Registration in `main.py`

```python
# backend/app/main.py

from fastapi import FastAPI
from app.routers import (
    assets_router,
    portfolios_router,
    transactions_router,
    upload_router,
)

app = FastAPI(title="Investment Portfolio Analyzer")

# Register all routers
app.include_router(assets_router)       # /assets/*
app.include_router(portfolios_router)   # /portfolios/*
app.include_router(transactions_router) # /transactions/*
app.include_router(upload_router)       # /upload/*

@app.get("/")
def root():
    return {"message": "Welcome!", "docs": "/docs"}
```

<a id='9-starting-server'></a>
# 9️⃣ Starting the Server

Let's start everything up!

In [ ]:
# Start Docker containers

display(Markdown("### 🚀 Starting Docker Containers"))

show_info("This may take 30-60 seconds on first run...")
show_warning("If containers are already running, this will restart them.")

print("\n1. Stopping existing containers...")
run_command("docker compose down", show_output=False)

print("2. Starting containers (with rebuild)...")
result = run_command("docker compose up -d --build", show_output=False)

if result.returncode == 0:
    show_success("Containers started!")
else:
    show_error(f"Failed: {result.stderr}")

In [ ]:
# Wait for API to be ready

display(Markdown("### ⏳ Waiting for API"))

def wait_for_api(max_attempts=30, delay=2):
    for attempt in range(max_attempts):
        try:
            response = requests.get(f"{API_URL}/", timeout=5)
            if response.status_code == 200:
                return True, response.json()
        except:
            pass
        print(f"  Attempt {attempt + 1}/{max_attempts}...")
        time.sleep(delay)
    return False, None

ready, info = wait_for_api()

if ready:
    show_success("API is ready!")
    print(f"\n📄 API Documentation: {API_URL}/docs")
    print(f"📊 API Info: {json.dumps(info, indent=2)}")
else:
    show_error("API failed to start. Run: docker compose logs backend")

In [ ]:
# Initialize database tables

display(Markdown("### 🗄️ Initializing Database"))

result = run_command("docker compose exec -T backend python init_db.py", show_output=True)

if result.returncode == 0:
    show_success("Database tables created!")
else:
    show_warning("Database may already be initialized (this is OK)")

In [ ]:
# Verify everything is working

display(Markdown("### ✅ System Health Check"))

# Check API root
print("1. Checking API...")
response = requests.get(f"{API_URL}/")
print(f"   Status: {response.status_code} {'✅' if response.status_code == 200 else '❌'}")

# Check database
print("2. Checking Database...")
response = requests.get(f"{API_URL}/health/db")
print(f"   Status: {response.status_code} {'✅' if response.status_code == 200 else '❌'}")

# Check endpoints
print("3. Checking Endpoints...")
endpoints = ["/assets/", "/portfolios/", "/transactions/", "/upload/formats"]
for ep in endpoints:
    try:
        r = requests.get(f"{API_URL}{ep}")
        print(f"   {ep}: {r.status_code} {'✅' if r.status_code == 200 else '❌'}")
    except:
        print(f"   {ep}: Error ❌")

show_success("System is ready!")

### 📝 Open Your Log Terminal Now!

Before continuing to Phase 2, open a new terminal and run:

```bash
docker compose logs -f backend
```

Keep it visible as you work through the rest of this notebook!

---

# 🔷 PHASE 2: DATA INGESTION

---

<a id='10-yahoo'></a>
# 🔟 Yahoo Finance Integration

How we fetch real market data.

## Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│                   MARKET DATA ARCHITECTURE                      │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌─────────────────┐                                           │
│  │ MarketDataProvider │◀─── Abstract base class                │
│  │ (base.py)         │     Defines interface                   │
│  └────────┬──────────┘                                         │
│           │                                                    │
│           │ implements                                         │
│           ▼                                                    │
│  ┌─────────────────┐     ┌─────────────────┐                  │
│  │YahooFinanceProvider│──▶│  Yahoo Finance  │                  │
│  │ (yahoo.py)        │   │  API (yfinance) │                  │
│  └────────┬──────────┘     └─────────────────┘                  │
│           │                                                    │
│           │ returns                                            │
│           ▼                                                    │
│  ┌─────────────────┐                                           │
│  │ AssetMetadata   │  name, sector, currency, asset_class     │
│  └─────────────────┘                                           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Why This Design?

- **Abstraction**: Can swap Yahoo for another provider
- **Testability**: Can mock the provider in tests
- **Extensibility**: Add more providers easily

## Provider Code

```python
# backend/app/services/market_data/yahoo.py

import yfinance as yf
from app.services.market_data.base import MarketDataProvider, AssetMetadata

class YahooFinanceProvider(MarketDataProvider):
    """Fetches data from Yahoo Finance."""
    
    def get_asset_metadata(self, ticker: str, exchange: str) -> AssetMetadata:
        """Fetch asset information from Yahoo."""
        
        # Build Yahoo symbol (e.g., "NVDA" or "BMW.DE")
        yahoo_symbol = self._build_symbol(ticker, exchange)
        
        # Fetch from Yahoo
        stock = yf.Ticker(yahoo_symbol)
        info = stock.info
        
        # Extract and return metadata
        return AssetMetadata(
            ticker=ticker,
            exchange=exchange,
            name=info.get('longName') or info.get('shortName'),
            asset_class=self._determine_asset_class(info),
            sector=info.get('sector'),
            industry=info.get('industry'),
            currency=info.get('currency', 'USD'),
            data_source='yahoo_finance',
        )
```

<a id='11-asset-resolution'></a>
# 1️⃣1️⃣ Asset Resolution Service

The **core innovation** of Phase 2.

## The Flow

```
┌─────────────────────────────────────────────────────────────────────┐
│                     ASSET RESOLUTION FLOW                           │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   INPUT: ticker="NVDA", exchange="NASDAQ"                          │
│                    │                                                │
│                    ▼                                                │
│   ┌────────────────────────────────────────┐                       │
│   │  1. Check Database                      │                       │
│   │     "SELECT * FROM assets               │                       │
│   │      WHERE ticker='NVDA'                │                       │
│   │      AND exchange='NASDAQ'"             │                       │
│   └─────────────────┬──────────────────────┘                       │
│                     │                                               │
│         ┌───────────┴───────────┐                                  │
│         │                       │                                  │
│         ▼                       ▼                                  │
│   ┌──────────┐           ┌──────────────────────────┐             │
│   │  FOUND   │           │  NOT FOUND               │             │
│   │          │           │                          │             │
│   │  Return  │           │  2. Call Yahoo Finance   │             │
│   │  Asset   │           │     Fetch metadata       │             │
│   │          │           │                          │             │
│   └──────────┘           │  3. Create Asset in DB   │             │
│                          │     INSERT INTO assets   │             │
│                          │                          │             │
│                          │  4. Return new Asset     │             │
│                          └──────────────────────────┘             │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# First, let's create a user and portfolio (needed for transactions)

display(Markdown("### 👤 Creating Test User & Portfolio"))

# Create user in database
create_user_code = '''
from app.database import SessionLocal
from app.models import User
db = SessionLocal()
try:
    existing = db.query(User).filter_by(email="demo@example.com").first()
    if existing:
        print(f"USER_ID:{existing.id}")
    else:
        user = User(email="demo@example.com", hashed_password="demo")
        db.add(user)
        db.commit()
        print(f"USER_ID:{user.id}")
finally:
    db.close()
'''
result = run_command(f'docker compose exec -T backend python -c "{create_user_code}"', show_output=False)
user_id = int(result.stdout.split(":")[1].strip()) if "USER_ID" in result.stdout else 1
print(f"User ID: {user_id}")

# Create portfolio
portfolio_data = {"name": "Demo Portfolio", "user_id": user_id, "currency": "EUR"}
response = requests.post(f"{API_URL}/portfolios/", json=portfolio_data)

if response.status_code == 201:
    portfolio_id = response.json()['id']
    show_success(f"Created portfolio ID: {portfolio_id}")
else:
    # Try to get existing
    response = requests.get(f"{API_URL}/portfolios/?user_id={user_id}")
    portfolio_id = response.json()['items'][0]['id'] if response.json()['items'] else 1
    show_info(f"Using existing portfolio ID: {portfolio_id}")

print(f"\n📁 Portfolio ID: {portfolio_id}")

In [ ]:
# Check current assets (should be empty or minimal)

display(Markdown("### 📊 Current Assets in Database"))

response = requests.get(f"{API_URL}/assets/")
assets = response.json()

print(f"Total assets: {assets['total']}")
if assets['items']:
    df = pd.DataFrame(assets['items'])
    display(df[['id', 'ticker', 'exchange', 'name', 'asset_class']])
else:
    show_info("No assets yet - perfect! We'll see them get created.")

<a id='12-transactions'></a>
# 1️⃣2️⃣ Creating Transactions

Let's see asset resolution in action!

In [ ]:
# Interactive Transaction Creator

display(Markdown("### 💰 Create Transaction"))

display(HTML("""
<div style="background:#fff3cd;padding:15px;border-radius:8px;margin-bottom:15px;">
    <strong>👀 WATCH YOUR TERMINAL!</strong><br>
    When you click "Create", watch the Docker logs for:<br>
    <code>INFO | Resolving asset: NVDA on NASDAQ</code><br>
    <code>INFO | Asset not in DB, fetching from provider...</code>
</div>
"""))

# Widgets
w_ticker = widgets.Text(value='NVDA', description='Ticker:')
w_exchange = widgets.Dropdown(options=['NASDAQ', 'NYSE', 'XETRA', 'LSE', 'AEB'], value='NASDAQ', description='Exchange:')
w_action = widgets.Dropdown(options=['BUY', 'SELL'], value='BUY', description='Action:')
w_quantity = widgets.FloatText(value=10.0, description='Quantity:')
w_price = widgets.FloatText(value=550.00, description='Price:')
w_currency = widgets.Dropdown(options=['USD', 'EUR', 'GBP'], value='USD', description='Currency:')
w_button = widgets.Button(description='Create Transaction', button_style='success', icon='check')
w_output = widgets.Output()

def on_create(b):
    with w_output:
        clear_output()
        
        data = {
            "portfolio_id": portfolio_id,
            "ticker": w_ticker.value.upper(),
            "exchange": w_exchange.value,
            "transaction_type": w_action.value,
            "date": datetime.now().isoformat() + "Z",
            "quantity": str(w_quantity.value),
            "price_per_share": str(w_price.value),
            "currency": w_currency.value,
            "fee": "0"
        }
        
        print(f"📤 POST {API_URL}/transactions/")
        print(f"   Ticker: {data['ticker']} on {data['exchange']}\n")
        
        response, elapsed = timed_request("POST", f"{API_URL}/transactions/", json=data)
        
        print(f"⏱️ Request took {elapsed:.2f} seconds")
        
        if response.status_code == 201:
            result = response.json()
            show_success(f"Transaction created! ID: {result['id']}")
            print(f"\n📈 Asset Details:")
            print(f"   Name: {result['asset']['name']}")
            print(f"   Sector: {result['asset'].get('sector', 'N/A')}")
            print(f"   Asset ID: {result['asset_id']}")
            
            if elapsed > 1.0:
                show_info("Slow response = Yahoo Finance was called (new asset)")
            else:
                show_info("Fast response = Asset was already in database")
        else:
            show_error(f"Failed: {response.status_code}")
            print(response.json())

w_button.on_click(on_create)

display(widgets.VBox([
    widgets.HBox([w_ticker, w_exchange]),
    widgets.HBox([w_action, w_quantity]),
    widgets.HBox([w_price, w_currency]),
    w_button,
    w_output
]))

In [ ]:
# Now create another transaction for the SAME ticker
# Watch how fast it is!

display(Markdown("### ⚡ Second Transaction (Cached Asset)"))

show_info("Creating another NVDA transaction - should be MUCH faster!")

data = {
    "portfolio_id": portfolio_id,
    "ticker": "NVDA",
    "exchange": "NASDAQ",
    "transaction_type": "BUY",
    "date": datetime.now().isoformat() + "Z",
    "quantity": "5",
    "price_per_share": "600.00",
    "currency": "USD",
    "fee": "0"
}

response, elapsed = timed_request("POST", f"{API_URL}/transactions/", json=data)

print(f"⏱️ Request took {elapsed:.2f} seconds")

if response.status_code == 201:
    show_success("Transaction created!")
    print("\n✨ No Yahoo call needed - asset was already in database!")

<a id='13-batch'></a>
# 1️⃣3️⃣ Batch Operations

In [ ]:
# Batch transaction creation

display(Markdown("### 📦 Batch Transaction Creation"))

batch_data = [
    {"portfolio_id": portfolio_id, "ticker": "AAPL", "exchange": "NASDAQ", 
     "transaction_type": "BUY", "date": "2024-06-01T10:00:00Z",
     "quantity": "20", "price_per_share": "195.00", "currency": "USD", "fee": "0"},
    {"portfolio_id": portfolio_id, "ticker": "MSFT", "exchange": "NASDAQ",
     "transaction_type": "BUY", "date": "2024-06-01T10:00:00Z",
     "quantity": "15", "price_per_share": "420.00", "currency": "USD", "fee": "0"},
    {"portfolio_id": portfolio_id, "ticker": "GOOGL", "exchange": "NASDAQ",
     "transaction_type": "BUY", "date": "2024-06-01T10:00:00Z",
     "quantity": "10", "price_per_share": "175.00", "currency": "USD", "fee": "0"},
]

print(f"Creating {len(batch_data)} transactions: AAPL, MSFT, GOOGL")
print("\n👀 Watch the logs for batch asset resolution!\n")

response, elapsed = timed_request("POST", f"{API_URL}/transactions/batch", json=batch_data)

print(f"⏱️ Batch took {elapsed:.2f} seconds")

if response.status_code == 201:
    result = response.json()
    show_success(f"Created {len(result)} transactions!")
    
    for txn in result:
        print(f"  ✓ {txn['asset']['ticker']}: {txn['quantity']} @ ${txn['price_per_share']}")

<a id='14-csv-upload'></a>
# 1️⃣4️⃣ CSV File Upload

In [ ]:
# Interactive CSV Upload

display(Markdown("### 📤 CSV File Upload"))

sample_csv = """date,action,ticker,reference_exchange,quantity,price,price_currency,fee,fee_currency
3/15/2024,Buy,AMZN,NASDAQ,5,175.50,USD,4.99,USD
3/20/2024,Buy,META,NASDAQ,8,485.00,USD,4.99,USD
4/01/2024,Buy,TSLA,NASDAQ,10,175.00,USD,0.00,USD"""

w_format = widgets.Dropdown(
    options=[('US (M/D/YYYY)', 'US'), ('EU (D/M/YYYY)', 'EU'), ('ISO (YYYY-MM-DD)', 'ISO')],
    value='US',
    description='Date Format:'
)
w_csv = widgets.Textarea(value=sample_csv, description='CSV:', layout=widgets.Layout(width='600px', height='150px'))
w_upload = widgets.Button(description='Upload CSV', button_style='primary', icon='upload')
w_upload_out = widgets.Output()

def do_upload(b):
    with w_upload_out:
        clear_output()
        
        with tempfile.NamedTemporaryFile(mode='w', suffix='.csv', delete=False) as f:
            f.write(w_csv.value)
            temp_path = f.name
        
        try:
            with open(temp_path, 'rb') as f:
                response, elapsed = timed_request(
                    "POST",
                    f"{API_URL}/upload/transactions",
                    files={'file': ('upload.csv', f, 'text/csv')},
                    params={'portfolio_id': portfolio_id, 'date_format': w_format.value}
                )
            
            print(f"⏱️ Upload took {elapsed:.2f} seconds\n")
            
            result = response.json()
            if result.get('success'):
                show_success(f"Created {result['created_count']} transactions!")
            else:
                show_error(f"{result['error_count']} errors")
                for err in result.get('errors', []):
                    print(f"  Row {err['row_number']}: {err['message']}")
        finally:
            os.unlink(temp_path)

w_upload.on_click(do_upload)

display(widgets.VBox([w_format, w_csv, w_upload, w_upload_out]))

<a id='15-errors'></a>
# 1️⃣5️⃣ Error Handling

In [ ]:
# Wrong date format demo

display(Markdown("### ❌ Wrong Date Format Demo"))

bad_csv = """date,action,ticker,reference_exchange,quantity,price,price_currency,fee,fee_currency
15/03/2024,Buy,JPM,NYSE,10,185.00,USD,0.00,USD"""

show_warning("Uploading EU dates (D/M/YYYY) but specifying US format (M/D/YYYY)")

with tempfile.NamedTemporaryFile(mode='w', suffix='.csv', delete=False) as f:
    f.write(bad_csv)
    temp_path = f.name

try:
    with open(temp_path, 'rb') as f:
        response = requests.post(
            f"{API_URL}/upload/transactions",
            files={'file': ('bad.csv', f, 'text/csv')},
            params={'portfolio_id': portfolio_id, 'date_format': 'US'}
        )
    
    result = response.json()
    if not result.get('success'):
        show_error("Upload failed as expected!")
        for err in result.get('errors', []):
            print(f"  Row {err['row_number']}: {err['message']}")
        
        show_info("'15/03/2024' can't be parsed as M/D/YYYY (no 15th month!)")
finally:
    os.unlink(temp_path)

<a id='16-exploration'></a>
# 1️⃣6️⃣ Data Exploration

In [ ]:
# Summary of all data

display(Markdown("## 📊 Portfolio Summary"))

# Get data
assets = requests.get(f"{API_URL}/assets/?limit=100").json()['items']
transactions = requests.get(f"{API_URL}/transactions/?portfolio_id={portfolio_id}&limit=100").json()['items']

# Stats
buys = sum(1 for t in transactions if t['transaction_type'] == 'BUY')
sells = sum(1 for t in transactions if t['transaction_type'] == 'SELL')

display(HTML(f"""
<div style="display:flex;gap:20px;margin-bottom:20px;">
    <div style="background:#e3f2fd;padding:20px;border-radius:8px;flex:1;text-align:center;">
        <h2 style="margin:0;color:#1976d2;">{len(assets)}</h2><p>Assets</p>
    </div>
    <div style="background:#e8f5e9;padding:20px;border-radius:8px;flex:1;text-align:center;">
        <h2 style="margin:0;color:#388e3c;">{len(transactions)}</h2><p>Transactions</p>
    </div>
    <div style="background:#fff3e0;padding:20px;border-radius:8px;flex:1;text-align:center;">
        <h2 style="margin:0;color:#f57c00;">{buys}</h2><p>Buys</p>
    </div>
    <div style="background:#fce4ec;padding:20px;border-radius:8px;flex:1;text-align:center;">
        <h2 style="margin:0;color:#c2185b;">{sells}</h2><p>Sells</p>
    </div>
</div>
"""))

In [ ]:
# Assets table

display(Markdown("### 📈 Assets"))

if assets:
    df = pd.DataFrame(assets)
    cols = ['ticker', 'exchange', 'name', 'asset_class', 'sector', 'currency']
    display(df[[c for c in cols if c in df.columns]])
else:
    show_info("No assets yet")

In [ ]:
# Transactions table

display(Markdown("### 💳 Transactions"))

if transactions:
    rows = []
    for t in transactions:
        rows.append({
            'Date': t['date'][:10],
            'Type': t['transaction_type'],
            'Ticker': t['asset']['ticker'],
            'Qty': float(t['quantity']),
            'Price': float(t['price_per_share']),
            'Total': float(t['quantity']) * float(t['price_per_share']),
            'Ccy': t['currency']
        })
    display(pd.DataFrame(rows))
else:
    show_info("No transactions yet")

In [ ]:
# Current holdings

display(Markdown("### 💼 Current Holdings"))

holdings = {}
for t in transactions:
    ticker = t['asset']['ticker']
    qty = float(t['quantity'])
    price = float(t['price_per_share'])
    
    if ticker not in holdings:
        holdings[ticker] = {'qty': 0, 'cost': 0, 'ccy': t['currency']}
    
    if t['transaction_type'] == 'BUY':
        holdings[ticker]['qty'] += qty
        holdings[ticker]['cost'] += qty * price
    else:
        holdings[ticker]['qty'] -= qty

rows = []
for ticker, data in holdings.items():
    if data['qty'] > 0:
        rows.append({
            'Ticker': ticker,
            'Shares': f"{data['qty']:.2f}",
            'Avg Cost': f"{data['cost']/data['qty']:.2f}" if data['qty'] > 0 else 'N/A',
            'Total Cost': f"{data['cost']:,.2f}",
            'Currency': data['ccy']
        })

if rows:
    display(pd.DataFrame(rows))
else:
    show_info("No open positions")

<a id='17-debugging'></a>
# 1️⃣7️⃣ Debugging & Logs

## Log Commands Reference

| Command | Purpose |
|---------|--------|
| `docker compose logs backend` | View all backend logs |
| `docker compose logs -f backend` | Follow logs in real-time |
| `docker compose logs --tail=50 backend` | Last 50 lines |
| `docker compose logs --since="5m" backend` | Last 5 minutes |
| `docker compose logs db` | Database logs |

In [ ]:
# View recent logs

display(Markdown("### 📜 Recent Backend Logs"))

result = run_command("docker compose logs --tail=30 backend", show_output=False)

display(HTML(f"""
<pre style="background:#1e1e1e;color:#d4d4d4;padding:15px;border-radius:8px;max-height:400px;overflow-y:auto;font-size:11px;">
{result.stdout}
</pre>
"""))

<a id='18-experiments'></a>
# 1️⃣8️⃣ Your Experiments

Try your own experiments!

In [ ]:
# EXPERIMENT: Try different tickers!
# Change these values and run

MY_TICKER = "DIS"  # Try: NFLX, TSLA, AMD, etc.
MY_EXCHANGE = "NYSE"  # Try: NASDAQ, XETRA
MY_QUANTITY = 10
MY_PRICE = 100.00

data = {
    "portfolio_id": portfolio_id,
    "ticker": MY_TICKER,
    "exchange": MY_EXCHANGE,
    "transaction_type": "BUY",
    "date": datetime.now().isoformat() + "Z",
    "quantity": str(MY_QUANTITY),
    "price_per_share": str(MY_PRICE),
    "currency": "USD",
    "fee": "0"
}

response = requests.post(f"{API_URL}/transactions/", json=data)

if response.status_code == 201:
    r = response.json()
    show_success(f"Created! {r['asset']['name']}")
else:
    show_error("Failed")
    print(response.json())

In [ ]:
# EXPERIMENT: Query the API
# Uncomment different requests to try them

# All assets
response = requests.get(f"{API_URL}/assets/")

# All transactions
# response = requests.get(f"{API_URL}/transactions/?portfolio_id={portfolio_id}")

# Single asset
# response = requests.get(f"{API_URL}/assets/1")

show_response(response)

---

# 🎉 Congratulations!

You've completed the full walkthrough!

## What You Learned

### Phase 1: Foundation
- ✅ Docker & containerization
- ✅ Project structure
- ✅ Configuration management
- ✅ Database connections
- ✅ SQLAlchemy models
- ✅ Pydantic schemas
- ✅ FastAPI routers

### Phase 2: Data Ingestion
- ✅ Yahoo Finance integration
- ✅ Asset resolution service
- ✅ Transaction management
- ✅ Batch operations
- ✅ CSV file upload
- ✅ Date format handling
- ✅ Error handling

## What's Next: Phase 3

- 📈 Market Data Engine (historical prices)
- 🔮 Proxy Backcasting (for data gaps)
- 📊 Portfolio valuation over time

---

## Quick Reference

| Endpoint | Method | Description |
|----------|--------|-------------|
| `/` | GET | API status |
| `/docs` | GET | API documentation |
| `/assets/` | GET | List assets |
| `/portfolios/` | GET, POST | Manage portfolios |
| `/transactions/` | GET, POST | Manage transactions |
| `/transactions/batch` | POST | Batch create |
| `/upload/transactions` | POST | CSV upload |

**Full docs:** http://localhost:8000/docs